In [73]:
import os
import json
import pandas as pd

from pycocotools.coco import COCO

In [74]:
json_path = './dataset/test.json'
coco = COCO(json_path)

# json load
with open(json_path) as f: coco_json = json.load(f)

# submission csv -> dataframe
data = pd.read_csv('./submission.csv')


loading annotations into memory...
Done (t=0.27s)
creating index...
index created!


In [75]:
images = []
annotations = []
anno_cnt = 0
for i in range(len(data)):
    PredictionString, image_id  = data.iloc[i]
    file_num = int(os.path.basename(image_id).split('.')[0])
    
    # image
    images.append(coco_json['images'][file_num])
    
    # annotation
    try:
        pred = PredictionString.split()
    except:
        print('non pred')
    
    for idx in range(0, len(pred), 6):
        label = int(pred[idx])
        score, x_min, y_min, x_max, y_max = map(float, pred[idx+1:idx+6])
        annotations.append({
            "image_id": file_num,
            "category_id": label,
            "area": round((x_max-x_min)*(y_max-y_min), 2),
            "bbox": [
                round(x_min, 1),
                round(y_min, 1),
                round(x_max-x_min, 1),
                round(y_max-y_min, 1)
            ],
            "iscrowd": 0,
            "id": anno_cnt
        })
        anno_cnt += 1
    

non pred
non pred
non pred
non pred
non pred


In [76]:
result = dict()
result['info'] = coco_json['info']
result['licenses'] = coco_json['licenses']
result['categories'] = coco_json['categories']
result['images'] = images
result['annotations'] = annotations

In [77]:
with open('csv2json.json', 'w') as outfile:
    json.dump(result, outfile, indent=4)